데이터셋은 우선적으로는 CelebDFV2를 사용하고, 이후에 GTA-V 데이터를 합쳐보도록 하자.

이걸 어디부터 구현해야 할까...

일단 SigLIP을 불러오는 것부터 시작하자. Quantize를 해줘서 최대한 부담을 줄여주자.

# 필요한 라이브러리 설치 & 초기화

In [1]:
!uv pip install --system flash-attn --no-build-isolation
!pip install -q flash-attn

Using Python 3.12.12 environment at: /usr
Audited 1 package in 202ms


In [2]:
!uv pip install --system lightning
!pip install -q lightning

Using Python 3.12.12 environment at: /usr
Audited 1 package in 103ms


In [3]:
!uv pip install --system jaxtyping
!pip install -q jaxtyping

Using Python 3.12.12 environment at: /usr
Audited 1 package in 98ms


In [4]:
!uv pip install --system bitsandbytes
!pip install -q bitsandbytes

Using Python 3.12.12 environment at: /usr
Audited 1 package in 96ms


In [5]:
!uv pip install --force-reinstall torchcodec==0.9.0 --index-url=https://download.pytorch.org/whl/cu126

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1.01s
Prepared 1 package in 1ms
Uninstalled 1 package in 11ms
Installed 1 package in 11ms
 ~ torchcodec==0.9.0+cu126


In [6]:
import os
# 컴파일 워커를 2개 정도로 제한 (전체 12개 중 일부만 사용)
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "2"
# os.environ["TORCH_COMPILE_DEBUG"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [7]:
import torch
torch.set_float32_matmul_precision('high')

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [8]:
import lightning.pytorch as L
from PIL import Image
from transformers import AutoProcessor, AutoModel, BitsAndBytesConfig
from pydantic import BaseModel

embed_size = 768

frame_token = 576

솔직히, 왠만하면 원본 모델을 따라가고 싶지만... 일단 인코더를 더 경량으로 바꾼다.

나중에 성능 안 나오면 탓할 것 중 인코더가 늘었다.

In [9]:
import torch.nn as nn

# 모델 구현

## 인코더

In [10]:
from jaxtyping import Float, Int, Bool
from torch import Tensor
from typing import overload, cast, Literal

In [11]:
import torch.nn.functional as F
from flash_attn import flash_attn_func

class ViTEncoder(nn.Module):
    def __init__(self, embed_size=768, num_heads=12, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads
        assert self.head_dim * num_heads == embed_size, "embed_size must be divisible by num_heads"

        self.q_proj = nn.Linear(embed_size, embed_size)
        self.k_proj = nn.Linear(embed_size, embed_size)
        self.v_proj = nn.Linear(embed_size, embed_size)
        self.out_proj = nn.Linear(embed_size, embed_size)
        self.dropout_p = dropout

        self.ln1 = nn.LayerNorm(embed_size)
        self.mlp = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(embed_size)

    @overload
    def forward(
            self, x: Float[Tensor, "batch seq_len embed_size"], return_attn_output: Literal[False]
    ) -> Float[Tensor, "batch seq_len embed_size"]:...

    @overload
    def forward(
            self, x: Float[Tensor, "batch seq_len embed_size"], return_attn_output: Literal[True]
    ) -> tuple[Float[Tensor, "batch seq_len embed_size"], Float[Tensor, "batch seq_len heads h_dim"]]:...

    def forward(self, x: Float[Tensor, "b l s"], return_attn_output=False):
        # x: (batch, token/frame * frames, embed_size)
        batch_size, frame_token, embed_size = x.shape

        # Project queries, keys, values
        q = self.q_proj(x) # (batch, token/frame * frames, embed_size)
        k = self.k_proj(x) # (batch, token/frame * frames, embed_size)
        v = self.v_proj(x) # (batch, token/frame * frames, embed_size)

        # Split into multiple heads
        # (batch, token/frame * frames, num_heads, head_dim)
        q: Float[Tensor, "b l h d"] = q.view(batch_size, frame_token, self.num_heads, self.head_dim)
        k: Float[Tensor, "b l h d"] = k.view(batch_size, frame_token, self.num_heads, self.head_dim)
        v: Float[Tensor, "b l h d"] = v.view(batch_size, frame_token, self.num_heads, self.head_dim)

        # Apply scaled dot product attention
        # dropout_p는 훈련 중일 때만 적용
        attn_output_raw = cast(Float[Tensor, "b l h d"], flash_attn_func(
            q, k, v,
            dropout_p=self.dropout_p if self.training else 0.0,
            softmax_scale=None, # None이면 1/sqrt(head_dim) 자동 적용
            causal=False
        ))
        # attn_output: (batch, token/frame * frames, num_heads, head_dim)

        # Concatenate heads and apply final linear projection
        attn_output: Float[Tensor, "b l s"] = attn_output_raw.contiguous().view(batch_size, frame_token, embed_size)
        attn_output = self.out_proj(attn_output) # (batch_size, seq_len, embed_size)


        x = self.ln1(x + attn_output) # Residual connection + LayerNorm
        x = self.ln2(x + self.mlp(x))

        if return_attn_output:
            return x, attn_output_raw
        return x

## Positional Encoding

In [12]:
import math


class TemporalPositionalEncoding(nn.Module):
    def __init__(self, embed_dim: int, num_frames: int = 32, dropout: float = 0.1):
        """
        Args:
            embed_dim: Feature dimension (ds = 768)
            num_frames: Maximum number of frames (nf = 32)
            dropout: Dropout probability
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 1. Frame Index (j) 생성: 0 ~ max_len-1
        # 논문의 'j'에 해당합니다.
        position: Float[Tensor, "frame 1"] = torch.arange(num_frames).unsqueeze(1).float()

        # 2. Div Term 계산 (논문 Eq 1의 분모 부분)
        # 10000^(2i/ds) 부분을 로그 스케일로 계산
        div_term: Float[Tensor, "frame embed_half"] = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))

        # 3. PE Matrix 초기화 [max_len, d_model]
        pe: Float[Tensor, "frame embed"] = torch.zeros(num_frames, embed_dim)

        # 4. 논문 수식 (Eq 1) 적용
        # PE(j, 2i) = sin(...) -> 짝수 인덱스
        pe[:, 0::2] = torch.sin(position * div_term)
        # PE(j, 2i+1) = cos(...) -> 홀수 인덱스
        pe[:, 1::2] = torch.cos(position * div_term)

        # 5. 차원 확장 (Broadcasting 준비)
        # PE는 [Frame, Dim] 정보를 담고 있습니다.
        # 입력 x: [Batch, Frames, Tokens, Dim]
        # PE  : [1,     Frames, 1,      Dim] 형태로 만들어야
        # Batch와 Tokens 차원으로 자동 확장(Broadcast)되어 더해집니다.
        pe: Float[Tensor, "1 frame 1 embed"] = pe.unsqueeze(0).unsqueeze(2)

        self.register_buffer('pe', pe)

    def forward(self, x: Float[Tensor, "batch frame token embed"]) -> Float[Tensor, "batch frame token embed"]:
        """
        Args:
            x: [Batch, Frames, Tokens, Dim]
               (예: [B, 32, 576, 768])
        Returns:
            x: Temporal PE가 더해진 텐서
        """
        self.pe: Float[Tensor, "1 frame 1 embed"]
        # 입력된 비디오의 실제 프레임 수만큼만 PE를 잘라서 사용
        current_frames = x.size(1)

        # x에 PE를 더함.
        # pe[:, :current_frames, :, :]의 shape은 [1, F, 1, D]
        # x의 shape [B, F, T, D]에 맞춰서,
        # 같은 프레임(F)에 있는 모든 토큰(T)들에게 동일한 PE 벡터가 더해짐.
        x = x + self.pe[:, :current_frames, :, :]

        return self.dropout(x)

## GPU Preprocessor
당장은 안 쓰이지만, 혹시나 필요하다면

In [13]:
import torch
from torchvision.transforms import v2
from transformers import Siglip2ImageProcessor, Siglip2Processor

class GPUSigLIPProcessor:
    def __init__(self, processor: Siglip2Processor):
        config = cast(Siglip2ImageProcessor, processor.image_processor)  # type: ignore

        # 1. 리사이즈 설정: Bilinear + Antialias=True가 핵심
        # Fast 프로세서가 텐서를 처리할 때 사용하는 로직과 일치시킵니다.
        self.resize = v2.Resize(
            size=(config.size['height'], config.size['width']),
            interpolation=v2.InterpolationMode.BILINEAR, # resample=2
            antialias=True # 오차를 줄이는 가장 중요한 설정
        )

        # 2. 정규화 설정
        # (x - 0.5) / 0.5 연산
        self.mean: Int[Tensor, "1 3 1 1"] = torch.tensor(config.image_mean).view(1, 3, 1, 1)
        self.std: Int[Tensor, "1 3 1 1"] = torch.tensor(config.image_std).view(1, 3, 1, 1)
        self.rescale_factor = config.rescale_factor

    def __call__(self, video_tensor: Float[Tensor, "batch 3 frame H W"]) -> Float[Tensor, "batch_frame 3 h w"]:
        """
        video_tensor: (batch, 3, frame, H, W), float32, GPU
        """
        b, c, t, h, w = video_tensor.shape
        device = video_tensor.device

        # 차원 변경 (B*T, C, H, W)
        x: Float[Tensor, "batch frame 3 H W"] = video_tensor.permute(0, 2, 1, 3, 4)
        x: Float[Tensor, "batch_frame 3 H W"] = x.flatten(0, 1)

        # [Step 1] Resize (uint8 상태에서 수행하거나 float32에서 수행)
        # torchvision v2는 uint8 입력을 받아 내부적으로 고정밀 연산을 수행합니다.
        x: Float[Tensor, "batch_frame 3 h w"] = self.resize(x)

        # [Step 2] Rescale (0~255 -> 0~1)
        x: Float[Tensor, "batch_frame 3 h w"] = x * self.rescale_factor

        # [Step 3] Normalize (x - 0.5) / 0.5
        # mean, std를 캐싱하여 속도 최적화
        self.mean = self.mean.to(device)
        self.std = self.std.to(device)
        x = (x - self.mean) / self.std

        # [Step 4] 최종 모델 입력형태인 float16으로 반환
        return x.to(torch.float16)

## UNITE

In [14]:
from typing import NamedTuple

class UNITEConfig(BaseModel):
    num_channel: int = 3
    num_cls: int = 2
    num_heads: int = 12
    num_frames: int = 32
    dropout: float = 0.1
    encoder_model: str = "google/siglip2-base-patch16-384"
    use_bfloat: bool = True
    cpu_preprocess: bool = True

class UNITEOutput(NamedTuple):
    res: Float[Tensor, "batch cls"]
    ad_param: Float[Tensor, "batch head frame"] | None = None
    embed: Float[Tensor, "batch embed"] | None = None

In [15]:
class UNITE(nn.Module):
    def __init__(self, config: UNITEConfig | None = None):
        super().__init__()

        if config is None:
            config = UNITEConfig()

        self.config = config

        dtype = torch.bfloat16 if config.use_bfloat else torch.float16
        self.vis_encoder = AutoModel.from_pretrained(
            config.encoder_model,
            device_map="auto",
            dtype=dtype,
            attn_implementation="flash_attention_2",
        )
        self.embed_size = self.vis_encoder.config.vision_config.hidden_size
        processor = AutoProcessor.from_pretrained(config.encoder_model, use_fast=True)
        self.processor = GPUSigLIPProcessor(processor)

        for para in self.vis_encoder.parameters():
            para.requires_grad = False
        self.vis_encoder.eval()

        self.class_token: Float[Tensor, "embed"] = nn.Parameter(torch.randn((self.embed_size,)), requires_grad=True)

        self.pos_embedding = TemporalPositionalEncoding(self.embed_size, config.num_frames, config.dropout)
        self.first_encoder = ViTEncoder(self.embed_size, config.num_heads, config.dropout)
        self.encoders = nn.ModuleList([ViTEncoder(self.embed_size, config.num_heads, config.dropout) for _ in range(3)])
        self.mlp_head = nn.Linear(self.embed_size, config.num_cls)


    def forward(self, x: Int[Tensor, "batch channel frame h w"], return_ad_param=False, return_embed=False) -> UNITEOutput:
        self.vis_encoder.eval()

        # Input: [batch, c, frame, h, w]
        b, _, f, *_ = x.shape

        with torch.no_grad():
            if self.config.cpu_preprocess:
                pixels_int: Int[Tensor, "bf channel h w"] = x.permute(0, 2, 1, 3, 4).flatten(0, 1)
                pixels: Float[Tensor, "bf channel h w"] = pixels_int.to(self.vis_encoder.dtype)
            else:
                pixels: Float[Tensor, "bf channel h w"] = self.processor(x)
            encoded = cast(Float[Tensor, "bf token embed"],
                self.vis_encoder.vision_model(pixel_values=pixels).last_hidden_state)
        encoded_reshape: Float[Tensor, "batch frame token embed"] = encoded.reshape(b, f, -1, self.embed_size)
        train_in: Float[Tensor, "batch frame token embed"] = self.pos_embedding(encoded_reshape)

        _, _, t, d = train_in.shape
        # Reshape for transformer
        # tot_token = frame * token
        train_reshape: Float[Tensor, "batch tot_token embed"] = train_in.reshape(b, t*f, d)
        cls_token: Float[Tensor, "batch 1 embed"] = self.class_token.view(1, 1, -1).expand(b, -1, -1)
        # tot_w_cls = tot_token + 1
        transform_in: Float[Tensor, "batch tot_w_cls embed"] = torch.cat([cls_token, train_reshape], dim=1)

        P: Float[Tensor, "batch head frame"] | None  = None
        transformer_out: Float[Tensor, "batch tot_w_cls embed"]

        if return_ad_param:
            transformer_out, attn_output = cast(
                tuple[Float[Tensor, "batch tot_w_cls embed"], Float[Tensor, "batch tot_w_cls head head_dim"]],
                self.first_encoder(transform_in, return_attn_output=True)
            ) # head_dim = embed // head

            attn_output: Float[Tensor, "batch head tot_token head_dim"]
            attn_output = attn_output.permute(0, 2, 1, 3)[:, :, 1:, :]

            train_in_reshape: Float[Tensor, "batch tot_token head head_dim"]
            train_in_reshape = train_in.reshape(b, -1, self.config.num_heads, d // self.config.num_heads)

            train_in_permute: Float[Tensor, "batch head tot_token head_dim"]
            train_in_permute = train_in_reshape.permute(0, 2, 1, 3)

            P_val: Float[Tensor, "batch head tot_token head_dim"] = attn_output * train_in_permute
            P_reshape: Float[Tensor, "batch head frame token head_dim"]
            P_reshape = P_val.reshape(b, self.config.num_heads, f, t, -1)
            P = P_reshape.mean(dim=(3, 4))
        else:
            transformer_out: Float[Tensor, "batch tot_w_cls embed"] = self.first_encoder(x)

        for encoder in self.encoders:
            transformer_out: Float[Tensor, "batch tot_w_cls embed"] = encoder(transformer_out)

        # Get only cls_token
        cls_token_out: Float[Tensor, "batch 1 embed"] = transformer_out[:, 0, :]
        embed: Float[Tensor, "batch embed"] = cls_token_out.reshape(b, -1)
        res: Float[Tensor, "batch cls"] = self.mlp_head(embed)
        ret = UNITEOutput(
            res=res,
            ad_param=P if return_ad_param else None,
            embed=embed if return_embed else None
        )
        return ret

## AD Loss

In [16]:
class ADLossConfig(BaseModel):
    num_cls: int = 2
    num_heads: int = 12
    num_frames: int = 32
    delta_within: tuple[float, float] = (0.01, -2.0)
    delta_between: float = 0.5
    eta: float = 0.05

In [17]:
class ADLoss(nn.Module):
    def __init__(self, config: ADLossConfig | None = None):
        super().__init__()
        if config == None:
            config = ADLossConfig()

        self.config = config
        # C shape: [num_classes, num_heads, max_len]
        # 논문 식(3)에 따라 센터를 각 클래스별로 유지해야 함
        # C = torch.zeros(num_cls, num_heads, max_len)
        # Try changing C to random normalization so that it might not collapse due to initial condition.
        C: Float[Tensor, "cls head frame"]
        C = torch.randn(config.num_cls, config.num_heads, config.num_frames)
        C = F.normalize(C, p=2, dim=2)
        self.register_buffer('C', C)

        delta_within: Float[Tensor, "cls"] = torch.tensor(config.delta_within) # [0.01, -2.0] (True, Fake)
        self.register_buffer("delta_within", delta_within)

    @overload
    def forward(
            self, P: Float[Tensor, "batch head frame"], labels: Int[Tensor, "batch"], log_detail: Literal[False]
        ) -> Float[Tensor, ""]: ...

    @overload
    def forward(
            self, P: Float[Tensor, "batch head frame"], labels: Int[Tensor, "batch"], log_detail: Literal[True]
        ) -> tuple[Float[Tensor, ""], Float[Tensor, ""], Float[Tensor, ""], Float[Tensor, ""]]: ...

    def forward(self, P: Float[Tensor, "batch head frame"], labels: Int[Tensor, "batch"], log_detail=False):
        """
        P: [batch, num_heads, max_len]
        labels: [batch] (Class indices)
        """
        self.C: Float[Tensor, "cls head frame"]
        self.delta_within: Float[Tensor, "cls"]
        device = P.device

        # 각 헤드의 특징 벡터(F)를 정규화
        P_norm: Float[Tensor, "batch head frame"] = F.normalize(P, p=2, dim=2)

        loss_between = torch.tensor(0.0, device=device)
        head_dist_mean = torch.tensor(0.0, device=device)
        valid_cls = 0

        # 1. 센터 업데이트 (현재 로직 유지하되 헤드별로 정규화 상태 유지)
        if self.training:
            for c in range(self.config.num_cls):
                mask: Bool[Tensor, "batch"] = (labels == c)
                if mask.any():
                    valid_cls += 1
                    batch_class_mean: Float[Tensor, "head frame"]
                    batch_class_mean = P_norm[mask].mean(dim=0)
                    with torch.no_grad():
                        # C[c]: [head frame]
                        self.C[c] = (1 - self.config.eta) * self.C[c] + self.config.eta * batch_class_mean.detach()

                    # Calculating between class loss here (eq. 5)
                    # Note that in paper it is loss between C and C, but I edited it to P and P
                    # because C is detached from graph thus cannot have valid gradient.
                    # So assume center of P is almost C.
                    dist_matrix: Float[Tensor, "head head"]
                    dist_matrix = torch.cdist(batch_class_mean, batch_class_mean, p=2)

                    mask: Bool[Tensor, "head head"] = torch.triu(
                        torch.ones(self.config.num_heads, self.config.num_heads, device=device),
                        diagonal=1
                    ).bool()
                    different_heads_dist: Float[Tensor, "n_pair"] = dist_matrix[mask]
                    head_dist_mean: Float[Tensor, ""]
                    head_dist_mean += different_heads_dist.mean()
                    loss_between: Float[Tensor, ""]
                    loss_between += torch.relu(self.config.delta_between - different_heads_dist).mean()

        if valid_cls > 0:
            head_dist_mean /= valid_cls
            loss_between /= valid_cls

        # 센터 정규화 (거리 계산 전 필수)
        C_norm: Float[Tensor, "cls head frame"] = F.normalize(self.C, p=2, dim=2)

        # --- 2. Within-class Loss (식 4) ---
        # 각 샘플과 자기 클래스 센터 사이의 거리
        # C_norm[labels]: [H, F]
        diff_within: Float[Tensor, "batch head frame"] = P_norm - C_norm[labels]
        # L2 Norm 계산 (헤드와 프레임 차원에 대해)
        dist_within = cast(
            Float[Tensor, "batch"],
            torch.linalg.vector_norm(diff_within, ord=2, dim=(1, 2))
        )
        # 각 샘플별 delta 적용
        # loss_within = torch.relu(dist_within - self.delta_within[labels]).mean()

        # 샘플 수와 상관없이 일정한 스케일 유지
        loss_sum = torch.tensor(0.0, device=device)
        for c in range(self.config.num_cls):
            mask: Bool[Tensor, "batch"] = (labels == c)
            if mask.any():
                # 해당 클래스 샘플들만의 평균을 구함
                class_loss = Float[Tensor, ""]
                class_loss = torch.relu(dist_within[mask] - self.delta_within[c]).mean()
                loss_sum += class_loss
        loss_within = loss_sum / self.config.num_cls # 클래스당 기여도를 1/N로 고정


        if log_detail:
            return loss_within + loss_between, loss_within, loss_between, head_dist_mean

        return loss_within + loss_between

# 학습 준비물

In [18]:
from torchmetrics.classification import Accuracy, AveragePrecision, Precision, Recall, ROC, AUROC
from torchmetrics import MetricCollection

## Pytorch Lightning 모듈

In [19]:
from typing import TypedDict, Literal, NotRequired, Annotated
from torchmetrics import ConfusionMatrix
from pydantic import Field, model_validator
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import numpy as np

class OptimizerConfig(BaseModel):
    lr: float = 1e-4
    decay_steps: int = 1000

class LossConfig(BaseModel):
    lambda_1: float = 0.5
    lambda_2: float = 0.5

class UNITEClassifierConfig(BaseModel):
    num_cls: int = 2
    num_heads: int = 12
    num_frames: int = 32
    unite_model: UNITEConfig = Field(default_factory=UNITEConfig)
    ad_loss: ADLossConfig = Field(default_factory=ADLossConfig)
    optim: OptimizerConfig = Field(default_factory=OptimizerConfig)
    loss: LossConfig = Field(default_factory=LossConfig)

    @model_validator(mode="after")
    def ensure_param(self):
        self.unite_model.num_cls = self.num_cls
        self.unite_model.num_heads = self.num_heads
        self.unite_model.num_frames = self.num_frames

        self.ad_loss.num_cls = self.num_cls
        self.ad_loss.num_heads = self.num_heads
        self.ad_loss.num_frames = self.num_frames

        return self

class LitUNITEClassifier(L.LightningModule):
    def __init__(self, config: UNITEClassifierConfig | None = None):
        super().__init__()
        if config is None:
            config = UNITEClassifierConfig()
        self.save_hyperparameters(config.model_dump(exclude={
            'unite_model': {'num_cls', 'num_heads', 'num_frames'},
            'ad_loss': {'num_cls', 'num_heads', 'num_frames'}
        }))

        self.config = config
        self.model = UNITE(config.unite_model)
        self.ce_loss = nn.CrossEntropyLoss()
        self.ad_loss = ADLoss(config.ad_loss)

        MetricType = TypedDict('MetricType', {
            "task": Literal['multiclass'] | Literal['binary'],
            "num_classes": NotRequired[int],
            "average": NotRequired[Literal['macro']],
        })

        metric_param: MetricType = {"task": "multiclass", "num_classes": config.num_cls, "average": "macro"}

        metrics = MetricCollection([
            Accuracy(**metric_param),
            AveragePrecision(**metric_param),
            Precision(**metric_param),
            Recall(**metric_param),
            AUROC(**metric_param),
        ])

        self.val_metrics = metrics.clone(prefix="val/")
        self.val_roc = ROC(**metric_param)
        self.test_metrics = metrics.clone(prefix="test/")
        self.test_roc = ROC(**metric_param)

        self.confmat = ConfusionMatrix(task='multiclass', num_classes=config.num_cls)

        self.class_names = ["Real", "Fake"] if config.num_cls == 2 else [f"Class {i}" for i in range(config.num_cls)]

    def forward(self, x):
        return self.model(x)

    def on_save_checkpoint(self, checkpoint):
        """체크포인트 저장 시 Frozen 파라미터(Backbone)를 제외합니다."""
        state_dict = checkpoint['state_dict']

        # 저장하지 않을 키 필터링 (vis_encoder 관련 키들 제거)
        # 키 이름은 모델 구조에 따라 'model.vis_encoder.'로 시작합니다.
        keys_to_remove = [k for k in state_dict.keys() if "model.vis_encoder" in k]

        for k in keys_to_remove:
            del state_dict[k]


    def on_load_checkpoint(self, checkpoint):
        """로드 시 체크포인트에 없는 백본 가중치를 현재 모델에서 복사해서 채워줌"""
        state_dict = checkpoint['state_dict']
        model_state_dict = self.state_dict()

        # 현재 모델(self.model.vis_encoder)은 이미 __init__에서
        # from_pretrained로 로드된 상태입니다.
        # 체크포인트에 없는 키(백본 가중치)들을 모델의 현재 가중치로 채워줍니다.
        for key in model_state_dict:
            if key not in state_dict:
                state_dict[key] = model_state_dict[key]

    def training_step(
            self,
            batch: tuple[Float[Tensor, "batch channel frame h w"], Float[Tensor, "batch"]],
            atch_idx: int
        ) -> Float[Tensor, ""]:
        x, y = batch
        logit, P, _ = cast(UNITEOutput, self.model(x, return_ad_param=True))
        assert P is not None
        loss_ad, within, between, head_dist_mean = cast(
            tuple[Float[Tensor, ""], Float[Tensor, ""], Float[Tensor, ""], Float[Tensor, ""]],
            self.ad_loss(P, y, log_detail=True)
        )
        loss_ce = cast(Float[Tensor, ""], self.ce_loss(logit, y))
        loss = loss_ce * self.config.loss.lambda_1 + loss_ad * self.config.loss.lambda_2
        c_magnitude = torch.norm(self.ad_loss.C, p=2, dim=-1).mean()


        self.log_dict({
            "train/loss_ad": loss_ad,
            "train/loss_ad/loss_within": within,
            "train/loss_ad/loss_between": between,
            "train/head_dist_mean": head_dist_mean,
            "train/loss_ce": loss_ce,
            "train/loss": loss,
            "train/C_magnitude": c_magnitude
        }, logger=True)


        return loss

    def on_validation_epoch_start(self):
        self.val_preds = []
        self.val_labels = []
        self.val_embeds = []
        self.val_ps = []


    def validation_step(self, batch, batch_idx):
        x, y = batch
        logit, P, embed = self.model(x, return_ad_param=True, return_embed=True)
        loss_ad = self.ad_loss(P, y)
        loss_ce = self.ce_loss(logit, y)
        loss = loss_ce * self.config.loss.lambda_1 + loss_ad * self.config.loss.lambda_2
        self.log("val/loss_ad", loss_ad, logger=True)
        self.log("val/loss_ce", loss_ce, logger=True)
        self.log("val/loss", loss, prog_bar=True, logger=True)

        self.val_metrics.update(logit, y)
        self.log_dict(self.val_metrics, logger=True)


        # 데이터 수집 (CPU로 이동하여 메모리 절약)
        preds = torch.argmax(logit, dim=1)
        self.val_roc.update(logit, y)


        self.val_preds.append(preds.detach().cpu())
        self.val_labels.append(y.detach().cpu())
        self.val_embeds.append(embed.detach().cpu()) # [Batch, 768]
        self.val_ps.append(F.normalize(P, p=2, dim=2).detach().cpu())

    def on_validation_epoch_end(self):
        # 모든 배치 데이터 병합
        all_preds = torch.cat(self.val_preds)
        all_labels = torch.cat(self.val_labels)
        all_embeds = torch.cat(self.val_embeds)
        all_ps = torch.cat(self.val_ps)

        # 1. Confusion Matrix 시각화 및 로그
        self._log_confusion_matrix(all_preds, all_labels, stage="val")

        # 2. t-SNE 시각화 및 로그
        self._log_tsne(all_embeds, all_labels, predicts=all_preds, stage="val")

        fig, ax = self.val_roc.plot()
        if self.logger:
            self.logger.experiment.log({"val/ROC_curve": wandb.Image(fig)})
        self.val_roc.reset()
        plt.close(fig)

        # 3. C heatmap
        for cls in range(self.config.num_cls):
            cls_C_norm = F.normalize(self.ad_loss.C[cls], p=2, dim=-1)
            sim_matrix = torch.mm(cls_C_norm, cls_C_norm.t())
            heatmap = go.Heatmap(
                x=[f"Head_{i}" for i in range(self.config.num_heads)],
                y=[f"Head_{i}" for i in range(self.config.num_heads)],
                z=sim_matrix.cpu().detach().numpy(),
            )
            fig = go.Figure([heatmap])

            self.logger.experiment.log({
                f"val/center_sim_mat_{self.class_names[cls]}": wandb.Plotly(fig)
            })

        # 4. t-SNE of P and C
        # 데이터 준비
        tsne_embeds = []
        tsne_labels = []     # Real/Fake 구분용
        tsne_heads = []      # Head ID 구분용 (색상)
        tsne_types = []      # Center vs Sample 구분용 (크기)

        # [Centers 추출]
        for c_idx in range(self.config.num_cls):
            for h_idx in range(self.config.num_heads):
                c_vec = self.ad_loss.C[c_idx, h_idx].detach().cpu().numpy()
                tsne_embeds.append(c_vec)
                tsne_labels.append(self.class_names[c_idx]) # "Real" or "Fake"
                tsne_heads.append(h_idx)
                tsne_types.append("Center")

        # [Samples (P) 추출 및 샘플링]
        num_total_samples = all_ps.shape[0]
        sample_size = min(num_total_samples, 400) # 브라우저 부하 방지를 위해 적절히 조절
        indices = torch.randperm(num_total_samples)[:sample_size]

        for i in indices:
            label_name = self.class_names[int(all_labels[i].item())]
            for h_idx in range(self.config.num_heads):
                p_vec = all_ps[i, h_idx].detach().cpu().numpy()
                tsne_embeds.append(p_vec)
                tsne_labels.append(label_name)
                tsne_heads.append(h_idx)
                tsne_types.append("Sample")

        # t-SNE 실행
        tsne_embeds = np.array(tsne_embeds)
        tsne_2d = TSNE(n_components=2, random_state=42).fit_transform(tsne_embeds)

        # 시각화 시작
        fig, ax = plt.subplots(figsize=(10, 8))

        # 설정: Head별 색상, Class별 마커
        cmap = sns.color_palette("husl", self.config.num_heads)
        marker_map = {self.class_names[0]: 'o', self.class_names[1]: 'X'} # 0: Real(Circle), 1: Fake(X)

        for i in range(len(tsne_2d)):
            h_idx = tsne_heads[i]
            label = tsne_labels[i]
            m_type = tsne_types[i]

            color = cmap[h_idx]
            marker = marker_map.get(label, 'd')
            size = 150 if m_type == "Center" else 50
            alpha = 1.0 if m_type == "Center" else 0.4
            edge = 'black' if m_type == "Center" else 'none'
            zorder = 3 if m_type == "Center" else 2

            ax.scatter(
                tsne_2d[i, 0], tsne_2d[i, 1],
                c=[color], marker=marker, s=size,
                alpha=alpha, edgecolors=edge, linewidths=1.5, zorder=zorder
            )

        # 범례 설정 (Custom Legend)
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', label='Real', markerfacecolor='gray', markersize=10),
            Line2D([0], [0], marker='X', color='w', label='Fake', markerfacecolor='gray', markersize=10),
            Line2D([0], [0], marker='s', color='w', label='Center', markerfacecolor='none', markeredgecolor='black', markersize=12)
        ]
        # Head별 색상 범례 추가
        for h in range(self.config.num_heads):
            legend_elements.append(Line2D([0], [0], color=cmap[h], lw=3, label=f'Head {h}'))

        ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
        ax.set_title(f"t-SNE of P and C (Epoch {self.current_epoch})")
        plt.tight_layout()

        # WandB 로그
        if self.logger:
            self.logger.experiment.log({"val/P_C_tsne_plot": wandb.Image(fig)})

        plt.close(fig)


        # 메모리 해제 및 초기화
        self.val_preds.clear()
        self.val_labels.clear()
        self.val_embeds.clear()
        self.val_ps.clear() # 추가됨
        if hasattr(self, 'val_roc'):
            self.val_roc.reset()


    def on_test_epoch_start(self):
        self.test_preds = []
        self.test_labels = []
        self.test_embeds = []

    def test_step(self, batch, batch_idx):
        x, y = batch
        logit, embed = self.model(x, return_embed=True)
        preds = torch.argmax(logit, dim=1)

        self.test_preds.append(preds.detach().cpu())
        self.test_labels.append(y.detach().cpu())
        self.test_embeds.append(embed.detach().cpu())

        self.test_metrics.update(logit, y)
        self.log_dict(self.test_metrics, logger=True)

        self.test_roc.update(logit, y)

    def on_test_epoch_end(self):
        all_preds = torch.cat(self.test_preds)
        all_labels = torch.cat(self.test_labels)
        all_embeds = torch.cat(self.test_embeds)

        # 1. Confusion Matrix
        self._log_confusion_matrix(all_preds, all_labels, stage="test")

        # 2. t-SNE
        self._log_tsne(all_embeds, all_labels, predicts=all_preds, stage="test")

        fig, ax = self.test_roc.plot()
        if self.logger:
            self.logger.experiment.log({"test/ROC_curve": wandb.Image(fig)})
        plt.close(fig)
        self.test_roc.reset()

        self.test_preds.clear()
        self.test_labels.clear()
        self.test_embeds.clear()

    def _log_confusion_matrix(self, preds, labels, stage="val"):
        """Confusion Matrix를 그리고 WandB에 로그합니다."""
        conf_matrix = self.confmat(preds.to(self.device), labels.to(self.device))

        fig = plt.figure(figsize=(8, 6))
        sns.heatmap(
            conf_matrix.cpu().numpy(),
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=self.class_names,
            yticklabels=self.class_names
        )
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{stage.capitalize()} Confusion Matrix')

        # WandB 로그
        if self.logger:
            self.logger.experiment.log({f"{stage}/confusion_matrix": wandb.Image(fig)})
        plt.close(fig)

    def _log_tsne(self, embeds, labels, predicts: Tensor|list|None=None, stage="val", max_samples=2000):
        """t-SNE를 계산하고 WandB에 로그합니다. 데이터가 많으면 샘플링합니다."""

        # 데이터가 너무 많으면 t-SNE 속도가 매우 느려지므로 샘플링
        num_samples = embeds.shape[0]
        if num_samples > max_samples:
            indices = torch.randperm(num_samples)[:max_samples]
            embeds = embeds[indices]
            labels = labels[indices]
            if predicts is not None:
                predicts = predicts[indices]

        # 텐서를 numpy로 변환
        X = embeds.numpy()
        y = labels.numpy()

        # t-SNE 계산
        # perplexity는 데이터 수보다 작아야 함 (보통 30~50)
        n_components = 2
        perplexity = min(30, num_samples - 1)
        if perplexity < 5: return # 데이터가 너무 적으면 패스

        tsne = TSNE(n_components=n_components, perplexity=perplexity, max_iter=1000, random_state=42)
        X_embedded = tsne.fit_transform(X)

        # DataFrame 생성 (시각화 용이성 위해)
        df_tsne = pd.DataFrame(X_embedded, columns=['x', 'y'])
        df_tsne['label'] = [self.class_names[i] for i in y]

        style_col = None
        if predicts is not None:
            y_pred = predicts
            df_tsne['prediction'] = [self.class_names[i] for i in y_pred]
            style_col = 'prediction'

        # Plotting
        fig = plt.figure(figsize=(15, 12))
        sns.scatterplot(
            data=df_tsne, x='x', y='y', hue='label',
            style=style_col,
            palette=['blue', 'red'], # Real: Blue, Fake: Red
            markers={'Real': 'o', 'Fake': 'X'},
            alpha=0.7, s=100
        )
        plt.title(f'{stage.capitalize()} t-SNE Visualization')
        plt.grid(True, linestyle='--', alpha=0.3)

        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        plt.tight_layout()

        # WandB 로그
        if self.logger:
            self.logger.experiment.log({f"{stage}/t_sne": wandb.Image(fig)})
        plt.close(fig)

    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.model.parameters(), lr=self.config.optim.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optim, self.config.optim.decay_steps, gamma=0.5)

        return {
            "optimizer": optim,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

## 데이터셋

In [20]:
from typing import Callable

class DatasetConfig(BaseModel):
    num_frames: int = 32
    size: tuple[int, int] = (384, 384)
    device: str = 'cpu'
    encoder_model: str = "google/siglip2-base-patch16-384"
    cpu_preprocess: bool = True
    transform: Callable | None = None

In [21]:
import os
import math
import cv2
import torch
from pathlib import Path
from typing import Sequence, TypedDict
from collections import Counter
from torch.utils.data import Dataset, ConcatDataset
from torchvision.io import decode_image, read_file, ImageReadMode
from torchvision.transforms import v2
from torchcodec.decoders import VideoDecoder
import transformers
from transformers import BatchFeature

from abc import ABC, abstractmethod

class Sample(TypedDict):
    path: str
    chunk_idx: int
    label: int
    total_frames: int

class DeepFakeBaseDataset(Dataset, ABC):
    def __init__(self, paths: Sequence[Path | str], config: DatasetConfig | None = None):
        if config is None:
            config = DatasetConfig()
        self.config = config
        self.samples: list[Sample] = []

        print(f"Processing {len(paths)} paths...")
        self._prepare_samples(paths)
        print(f"Loaded {len(self.samples)} samples from {len(paths)} files/directories.")

        self.preprocessor = None
        if config.cpu_preprocess:
            self.preprocessor = transformers.AutoProcessor.from_pretrained(
                config.encoder_model,
                use_fast=True
            )

    @abstractmethod
    def _get_label(self, path: str) -> int | None:
        raise NotImplementedError

    def _prepare_samples(self, paths: Sequence[Path | str]):
        """상속받는 클래스에서 각자의 방식으로 samples 리스트를 채움"""
        for path in paths:
            path_str = str(path)
            label = self._get_label(path_str)
            if label is None:
                continue

            frame_cnt = self._get_frame_count(path_str)
            if frame_cnt <= 0:
                continue

            num_chunks = self._calculate_num_chunks(frame_cnt)
            for i in range(num_chunks):
                self.samples.append({
                    "path": path_str,
                    "chunk_idx": i,
                    "label": label,
                    "total_frames": frame_cnt
                })

    @abstractmethod
    def _get_frame_count(self, path: str) -> int:
        """자식 클래스에서 구현: 전체 프레임 수 반환"""
        raise NotImplementedError

    @abstractmethod
    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        """자식 클래스에서 구현: 프레임 수에 따른 청크 개수 계산"""
        raise NotImplementedError

    def get_label_counter(self) -> Counter:
        return Counter([x['label'] for x in self.samples])

    def __len__(self):
        return len(self.samples)

    @abstractmethod
    def __getitem__(self, idx: int) -> tuple[Float[Tensor, "channel batch h w"], Int[Tensor, "batch"]]:
        """자식 클래스에서 구현: 실제 데이터 로드"""
        raise NotImplementedError


### Celeb-DF

In [22]:
class CelebDFBaseDataset(DeepFakeBaseDataset, ABC):
    """
    Celeb-DF 데이터셋의 공통 기능을 담은 부모 클래스
    """
    def _get_label(self, path: str) -> int | None:
        """폴더명 기반 레이블 결정"""
        rel_path = path.replace("\\", "/")
        if "YouTube-real" in rel_path or "Celeb-real" in rel_path:
            return 0
        elif "Celeb-synthesis" in rel_path:
            return 1
        return None

class CelebDFVideoDataset(CelebDFBaseDataset):
    """
    비디오 파일(.mp4 등)에서 직접 프레임을 추출하는 데이터셋
    """
    def _get_frame_count(self, path: str) -> int:
        cap = cv2.VideoCapture(path)
        cnt = 0
        if cap.isOpened():
            cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
        return cnt

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        # Stride=2를 고려한 유효 프레임 기반 계산
        effective_frames = math.ceil(frame_cnt / 2)
        return math.ceil(effective_frames / self.config.num_frames)

    def __getitem__(self, idx: int):
        meta = self.samples[idx]
        video_path, chunk_idx, label = meta["path"], meta["chunk_idx"], meta["label"]

        try:
            decoder = VideoDecoder(video_path, device='cpu')
            total_frames = decoder.metadata.num_frames or 100000

            # Stride 2 적용하여 인덱스 계산
            start_frame = chunk_idx * self.config.num_frames * 2
            indices = [min(start_frame + (i * 2), total_frames - 1) for i in range(self.config.num_frames)]

            frames_batch = decoder.get_frames_at(indices=indices)
            pixel_value_raw: Int[Tensor, "batch channel h w"] = frames_batch.data

            pixel_value: Float[Tensor, "batch channel h w"]
            if self.config.transform:
                # Video transform이 필요한 경우 여기서 처리 (Batch 단위 지원 필요)
                pixel_value = cast(
                    Float[Tensor, "batch channel h w"],
                    self.config.transform(pixel_value_raw)
                )
            else:
                pixel_value = pixel_value_raw.to(torch.float32)

            frames_tensor: Float[Tensor, "batch channel h w"]
            if self.config.cpu_preprocess:
                assert self.preprocessor is not None
                processed = cast(
                    BatchFeature,
                    self.preprocessor(images=pixel_value, return_tensors="pt")
                )
                frames_tensor = cast(
                    Float[Tensor, "batch channel h w"],
                    processed.pixel_values
                )
            else:
                frames_tensor = pixel_value

            # (N, C, H, W) -> (C, N, H, W)
            frames_tensor_out: Float[Tensor, "channel batch h w"] = frames_tensor.permute(1, 0, 2, 3)

        except Exception as e:
            print(f"Error loading {video_path}: {e}")
            frames_tensor_out: Float[Tensor, "channel batch h w"]
            frames_tensor_out = torch.zeros(
                (3, self.config.num_frames, self.config.size[1], self.config.size[0]),
                dtype=torch.float32
            )

        return frames_tensor_out.contiguous(), torch.tensor(label)

In [23]:
from typing import Callable

class ImageProcessor:
    def __init__(self, naming_fn:Callable[[int], str], size: tuple[int, int]):
        self.naming_fn = naming_fn
        self.size = size

    def get_frame_count(self, path: str) -> int:
        try:
            return len([f for f in os.listdir(path) if f.endswith(('.jpg', '.png', '.bmp'))])
        except Exception:
            return 0

    def calculate_num_chunks(self, dataset: DeepFakeBaseDataset, frame_cnt: int) -> int:
        return math.ceil(frame_cnt / dataset.config.num_frames)

    def getitem(self, dataset: DeepFakeBaseDataset, idx: int) -> tuple[Float[Tensor, "channel batch h w"], Int[Tensor, "batch"]]:
        meta = dataset.samples[idx]
        folder_path, chunk_idx, label, total_frames = meta["path"], meta["chunk_idx"], meta["label"], meta["total_frames"]

        frames_list: list[Float[Tensor, "channel h w"]] = []
        start_frame_idx = chunk_idx * dataset.config.num_frames

        for i in range(dataset.config.num_frames):
            current_idx = min(start_frame_idx + i, total_frames - 1)
            file_name = self.naming_fn(current_idx)
            img_path = os.path.join(folder_path, file_name)

            try:
                img_tensor_raw: Int[Tensor, "channel h w"] = decode_image(
                    read_file(img_path), mode=ImageReadMode.RGB
                )
                img_tensor_raw = v2.Resize(self.size)(img_tensor)
                img_tensor: Float[Tensor, "channel h w"]
                if dataset.config.transform:
                    img_tensor = dataset.config.transform(img_tensor_raw)
                else:
                    img_tensor = img_tensor_raw.to(torch.float32)
                frames_list.append(img_tensor)
            except Exception:
                frames_list.append(
                    torch.zeros(
                        (3, dataset.config.size[1], dataset.config.size[0]),
                        dtype=torch.float32
                    )
                )

        if frames_list:
            frames_tensor: Float[Tensor, "batch channel h w"] = torch.stack(frames_list, dim=0)
            if dataset.config.cpu_preprocess:
                assert dataset.preprocessor is not None
                processed = cast(
                    BatchFeature,
                    dataset.preprocessor(images=frames_tensor, return_tensors="pt")
                )
                frames_tensor = cast(
                    Float[Tensor, "batch channel h w"],
                    processed.pixel_values
                )
            frames_tensor_out: Float[Tensor, "channel batch h w"] = frames_tensor.permute(1, 0, 2, 3)
        else:
            frames_tensor_out = torch.zeros(
                (3, dataset.config.num_frames, dataset.config.size[1], dataset.config.size[0]),
                dtype=torch.float32
            )


        return frames_tensor_out.contiguous(), torch.tensor(label)

In [24]:
class CelebDFImageDataset(CelebDFBaseDataset):
    """
    이미지 파일이 저장된 폴더에서 프레임을 읽어오는 데이터셋
    """
    def __init__(self, paths: Sequence[Path | str], config: DatasetConfig | None = None):
        super().__init__(paths, config)
        self.processor = ImageProcessor(CelebDFImageDataset.idx_to_filename, self.config.size)

    @staticmethod
    def idx_to_filename(idx: int) -> str:
        return f"frame_{idx+1:06d}.jpg"

    def _get_frame_count(self, path: str) -> int:
        return self.processor.get_frame_count(path)

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        return self.processor.calculate_num_chunks(self, frame_cnt)

    def __getitem__(self, idx):
        return self.processor.getitem(self, idx)

In [25]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm
from shutil import copy2
import os

def resize_single_video(item):
    src_path, dst_path, size = item
    # 폴더가 없으면 생성
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    # FFmpeg 명령어 구성
    # -y: 기존 파일 덮어쓰기
    # -i: 입력 파일
    # -vf: 비디오 필터 (리사이즈)
    # -c:v libx264: H.264 코덱 사용
    # -crf 23: 일반적인 화질 설정 (낮을수록 고화질)
    # -preset veryfast: 속도 우선 인코딩
    # -an: 오디오 제거 (학습에 필요 없음, 용량 절감)
    cmd = [
        'ffmpeg', '-y',
        '-hwaccel', 'cuda',             # 디코딩 가속 유지
        '-i', str(src_path),
        # 성공했던 코드의 필터 형식을 그대로 사용 (flags=lanczos 등)
        '-vf', f"scale={size[0]}:{size[1]}:flags=lanczos",
        '-c:v', 'libx264',              # NVENC 대신 검증된 libx264 사용
        '-preset', 'fast',              # 속도 조절
        '-crf', '23',                   # 화질 설정
        '-pix_fmt', 'yuv420p',          # 재생 호환성을 위해 추가
        '-an',                          # 오디오 제거
        str(dst_path)
    ]

    # 실행 (로그는 숨김)
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # ... (cmd 설정 부분)


def preprocess_celebdf(src_root, dst_root, size=(384, 384), max_workers=8):
    src_root = Path(src_root)
    dst_root = Path(dst_root)

    if dst_root.exists():
        print(f"Target directory {dst_root} already exists. Skipping preprocessing.")
        return

    # 모든 mp4 파일 찾기
    video_files = list(src_root.glob("*/*.mp4"))
    tasks = []

    for src_path in video_files:
        rel_path = src_path.relative_to(src_root)
        dst_path = dst_root / rel_path
        if not dst_path.exists():
            tasks.append((src_path, dst_path, size))

    print(f"Starting preprocessing: {len(tasks)} videos...")

    # 멀티프로세싱으로 병렬 처리
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(resize_single_video, tasks, chunksize=4), total=len(tasks)))
    copy2(src_root / "List_of_testing_videos.txt", dst_root / "List_of_testing_videos.txt")

    print("Preprocessing completed.")

In [26]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm
from shutil import copy2
import os

def extract_frames_single_video(item):
    src_path, dst_dir, size = item

    if dst_dir.exists() and any(dst_dir.iterdir()):
        return

    dst_dir.mkdir(parents=True, exist_ok=True)

    output_pattern = str(dst_dir / "frame_%06d.jpg")

    # FFmpeg 명령어 변경
    # -vf "select='not(mod(n\,2))',scale=...":
    #   1. select: 짝수 프레임(0, 2, 4...)만 선택
    #   2. scale: 리사이즈
    # -vsync vfr: 선택된 프레임만 출력 (타임스탬프 유지를 위해 빈 프레임 생성 방지)
    cmd = [
        'ffmpeg', '-y',
        '-hwaccel', 'cuda',
        '-i', str(src_path),
        '-vf', rf"select='not(mod(n\,2))',scale={size[0]}:{size[1]}:flags=lanczos",
        '-vsync', 'vfr', # Variable Frame Rate: 선택된 프레임만 씀
        '-q:v', '2',
        output_pattern,
        '-threads', '1',
        '-hide_banner', '-loglevel', 'error'
    ]

    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def preprocess_celebdf_frames(src_root, dst_root, size=(384, 384), max_workers=8):
    src_root = Path(src_root)
    dst_root = Path(dst_root)

    video_files = list(src_root.glob("*/*.mp4"))
    tasks = []

    print(f"Scanning files...")
    for src_path in video_files:
        rel_path = src_path.relative_to(src_root)
        dst_dir = dst_root / rel_path.with_suffix('')

        if not dst_dir.exists() or not any(dst_dir.iterdir()):
            tasks.append((src_path, dst_dir, size))

    print(f"Starting frame extraction (Even frames only): {len(tasks)} videos...")

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(extract_frames_single_video, tasks, chunksize=1), total=len(tasks)))

    txt_src = src_root / "List_of_testing_videos.txt"
    if txt_src.exists():
        copy2(txt_src, dst_root / "List_of_testing_videos.txt")

    print("Preprocessing completed.")

### SAIL_VOS dataset

원본 논문은 SAIL\_VOS\_3D 데이터셋을 사용하였지만, 쉽사리 얻을 수 있는 데이터셋도 아니고 해서 대신 SAIL\_VOS로 만족하자.

In [27]:
class SailVosDataset(DeepFakeBaseDataset):
    def __init__(self, paths: Sequence[Path | str], config: DatasetConfig | None = None, ext: str=".png"):
        self.ext = ext
        if config is None:
            config = DatasetConfig()
        self.processor = ImageProcessor(self.idx_to_filename, config.size)

        super().__init__(paths, config)

    def idx_to_filename(self, idx: int) -> str:
        return f"{idx:06d}{self.ext}"

    def _get_label(self, path: str) -> int | None:
        return 1

    def _get_frame_count(self, path: str) -> int:
        return self.processor.get_frame_count(path)

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        return self.processor.calculate_num_chunks(self, frame_cnt)

    def __getitem__(self, idx):
        return self.processor.getitem(self, idx)

In [28]:
import os
import glob
import cv2
from multiprocessing import Pool, cpu_count
from pathlib import Path
from tqdm.notebook import tqdm

def process_single_image(args):
    """
    하나의 이미지를 처리하는 워커 함수입니다.
    multiprocessing에서 호출됩니다.

    Args:
        args (tuple): (source_path, target_path, target_size)
    """
    src_path, dst_path, target_size = args

    try:
        # 이미지 읽기
        img = cv2.imread(src_path)
        if img is None:
            return f"Error reading: {src_path}"

        # 리사이징 (너비, 높이)
        # 보간법은 cv2.INTER_LINEAR (기본값) 혹은 축소 시 cv2.INTER_AREA 추천
        resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)

        # 저장할 폴더가 없으면 생성 (Race condition 방지를 위해 exist_ok=True)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)

        # 이미지 저장
        cv2.imwrite(dst_path, resized_img)
        return None  # 성공 시 None 반환

    except Exception as e:
        return f"Exception at {src_path}: {str(e)}"

def preprocess_gta_v(source_dir, output_dir, target_size, num_workers=8):
    """
    메인 처리 함수
    """
    source_path = Path(source_dir)
    output_path = Path(output_dir)

    # 1. 파일 목록 스캔
    # 구조: source_dir/Images/video_seq/000000.bmp
    # 패턴: 모든 하위 폴더의 images 폴더 안의 bmp 파일들
    print(f"Scanning files in {source_dir}...")

    # glob 패턴: source_dir 아래 모든 폴더(*) 아래 images 폴더 아래 *.bmp
    # 혹은 재귀적으로 찾으려면 rglob 사용 가능. 여기서는 데이터셋 구조에 맞춰 명시적으로 찾습니다.
    # SAIL-VOS 구조: root -> Images -> vid_seq -> *.bmp
    all_files = list(source_path.glob("Images/*/*.bmp"))

    if not all_files:
        print("No files found. Please check the source directory structure.")
        return

    print(f"Found {len(all_files)} images. Preparing tasks...")

    # 2. 작업 리스트 생성
    tasks = []

    for src_file in all_files:
        src_p = Path(src_file)

        # 경로 파싱
        # src_p: .../ah_1_mcs_1/000000.bmp
        # file_name: 000000.bmp
        # video_seq_name: ah_1_mcs_1 (부모의 부모 폴더 이름)

        file_name = src_p.with_suffix(".png").name
        video_seq_name = src_p.parent.name

        # 목표 경로 생성: output_dir/ah_1_mcs_1/000000.bmp
        # (images 폴더 depth를 제거함)
        dst_p = output_path / video_seq_name / file_name

        tasks.append((str(src_p), str(dst_p), target_size))

    # 3. 멀티프로세싱 실행
    print(f"Starting processing with {num_workers} workers...")
    print(f"Target Size: {target_size}")

    with Pool(processes=num_workers) as pool:
        # tqdm을 사용하여 진행률 표시
        # imap_unordered가 리스트를 미리 만들지 않아 메모리 효율적이며 순서 상관없이 처리됨
        results = list(tqdm(pool.imap_unordered(process_single_image, tasks), total=len(tasks)))

    # 4. 결과 리포트
    errors = [res for res in results if res is not None]

    print("\nProcessing Complete.")
    if errors:
        print(f"{len(errors)} errors occurred:")
        for err in errors[:10]: # 처음 10개만 출력
            print(err)
        if len(errors) > 10:
            print("...")
    else:
        print("Successfully processed all images without errors.")

## 데이터 모듈

In [29]:
import os
import zipfile
from tqdm.notebook import tqdm

def copy_with_progress(src, dst):
    # 파일 크기 확인
    file_size = os.path.getsize(src)

    # 파일을 바이너리 모드로 열기
    with open(src, 'rb') as fsrc:
        with open(dst, 'wb') as fdst:
            # tqdm 설정 (단위는 바이트, 총 크기 지정)
            with tqdm(total=file_size, unit='B', unit_scale=True, desc=os.path.basename(src)) as pbar:
                while True:
                    # 1MB씩 읽어서 복사
                    chunk = fsrc.read(1024 * 1024)
                    if not chunk:
                        break
                    fdst.write(chunk)
                    pbar.update(len(chunk)) # 프로그래스바 업데이트


def unzip_by_size(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # 전체 파일의 압축 해제 후 총 용량 계산
        total_size = sum(file.file_size for file in zip_ref.infolist())

        with tqdm(total=total_size, unit='B', unit_scale=True, desc="해제 중") as pbar:
            for file_info in zip_ref.infolist():
                zip_ref.extract(file_info, extract_to)
                pbar.update(file_info.file_size) # 각 파일 용량만큼 게이지 증가



In [30]:
from torch.utils.data import WeightedRandomSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import kagglehub
import pandas as pd
from typing import TypedDict, Sequence, Literal, Any, NotRequired
from pydantic import BaseModel, Field
from torchvision.transforms import v2

class DataLoaderConfig(BaseModel):
    batch_size: int = 32
    num_workers: int = 8
    prefetch_factor: int | None = 1
    pin_memory: bool = True
    persistent_workers: bool = True

class DataModuleConfig(BaseModel):
    celeb_df_path: Path = Path("/content/preprocessed")
    from_img: bool = True
    val_split_ratio: float = 0.1
    use_gta_v: bool = True
    gta_v_path: Path = Path("/content/gta_v_preprocessed")
    seed: int | None = None
    do_preprocess: bool = True
    run_sample: int = 20000
    loader: DataLoaderConfig = Field(default_factory=DataLoaderConfig)
    dataset: DatasetConfig = Field(default_factory=DatasetConfig)

class DFDataModule(L.LightningDataModule):
    def __init__(self, config: DataModuleConfig | None = None):
        super().__init__()
        if config is None:
            config = DataModuleConfig()

        self.config = config
        self.save_hyperparameters(config.model_dump(exclude={'dataset': {'transform'}}))

        self.dataset_cls: type[DeepFakeBaseDataset] = CelebDFImageDataset
        self.dataset_preprocess_method = preprocess_celebdf_frames
        if config.from_img == False:
            self.dataset_cls = CelebDFVideoDataset
            self.dataset_preprocess_method = preprocess_celebdf

        class LoaderParam(TypedDict):
            batch_size: int
            num_workers: int
            pin_memory: bool
            persistent_workers: bool
            prefetch_factor: int | None

        self._loader_param = LoaderParam(**config.loader.model_dump())

    def prepare_data(self):
        path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
        if self.config.do_preprocess:
            self.dataset_preprocess_method(path, self.config.celeb_df_path, size=self.config.dataset.size)
        else:
            self.config.celeb_df_path = Path(path)

        if self.config.use_gta_v:
            if self.config.gta_v_path.exists():
                print(f"GTA V data exists in {self.config.gta_v_path}, skipping preprocessing.")
            else:
                if not Path('./mini-ref-sailvos.zip').exists():
                    print("get Sailvos dataset from drive")
                    # get SAIL-VOS
                    copy_with_progress(
                        "/content/drive/MyDrive/bootcamp_proj/final/mini-ref-sailvos.zip",
                        "./mini-ref-sailvos.zip"
                    )
                if not Path("./gta-v").exists():
                    unzip_by_size("mini-ref-sailvos.zip", "./gta-v")
                if self.config.do_preprocess:
                    preprocess_gta_v(Path("./gta-v/mini-ref-sailvos"), self.config.gta_v_path, self.config.dataset.size)
                else:
                    self.config.gta_v_path = Path("./gta-v/mini-ref-sailvos/Images")


    def _setup_gta_path(self) -> tuple[list[Path], list[Path], list[Path]]:
        all_folders_gta =  [
            Path(x) for x in self.config.gta_v_path.glob("*") if os.path.isdir(x)
        ]

        # 8:1:1 split
        train_vid, val_test_vid = train_test_split(
            all_folders_gta, test_size = 0.2,
            random_state=42,
            shuffle=True,
        )

        val_vid, test_vid = train_test_split(
            val_test_vid, test_size = 0.5,
            random_state=42,
            shuffle=True,
        )

        return train_vid, val_vid, test_vid


    def setup(self, stage=None):
        gta_train_path, gta_val_path, gta_test_path = None, None, None
        if self.config.use_gta_v:
            gta_train_path, gta_val_path, gta_test_path = self._setup_gta_path()

        test_val_config = self.config.dataset.model_copy()
        test_val_config.transform = None

        train_config = self.config.dataset.model_copy()

        if stage == "fit" or stage is None:
            # 1. 모든 비디오 폴더 경로 가져오기
            # 전처리된 이미지 폴더 구조: root/Celeb-real/id0_0000 ...
            # glob을 이용해 실제 폴더들을 다 찾습니다.

            all_folders = []
            if self.config.from_img:
                all_folders = [Path(x) for x in self.config.celeb_df_path.glob("*/*") if os.path.isdir(x)]
            else:
                all_folders = [Path(x) for x in self.config.celeb_df_path.glob("*/*.mp4")]

            # 2. Test Set 목록 로드 및 제외
            txt_path = self.config.celeb_df_path / "List_of_testing_videos.txt"
            test_df = pd.read_csv(txt_path, sep=" ", header=None, names=["label", "path"])
            test_paths_set = set()
            if self.config.from_img:
                # 확장자를 떼고 비교해야 함 (이미지 폴더명은 확장자가 없으므로)
                test_paths_set = set(
                    test_df["path"].apply(lambda x: str(self.config.celeb_df_path / Path(x).with_suffix('')).replace("\\", "/")).values
                )
            else:
                test_paths_set = set(test_df["path"].apply(lambda x: str(self.config.celeb_df_path / x)).replace("\\", "/").values)

            train_val_candidates = []
            for folder in all_folders:
                # 경로 정규화
                folder_str = str(folder).replace("\\", "/")
                # 테스트 셋에 포함되지 않은 것만 Train/Val 후보로 등록
                if folder_str not in test_paths_set:
                     train_val_candidates.append(folder_str)

            # 3. 비디오 단위로 Train / Val 분리 (가장 중요!)
            # 여기서 쪼개야 비디오 하나가 통째로 Train이나 Val 한쪽으로만 갑니다.
            train_videos, val_videos = train_test_split(
                train_val_candidates,
                test_size=self.config.val_split_ratio,
                random_state=42,
                shuffle=True
            )

            print(f"Total Train/Val Videos: {len(train_val_candidates)}")
            print(f"Split result -> Train Videos: {len(train_videos)}, Val Videos: {len(val_videos)}")

            # 4. 데이터셋 인스턴스 생성 (video_paths 주입)
            self.celebdf_train = self.dataset_cls(
                paths=train_videos,
                config=train_config,
            )

            self.celebdf_val = self.dataset_cls(
                paths=val_videos,
                config=test_val_config,
            )

            if self.config.use_gta_v:
                assert gta_train_path is not None
                assert gta_val_path is not None
                if not self.config.do_preprocess:
                    ext = ".bmp"
                else:
                    ext = ".png"
                self.gta_train = SailVosDataset(
                    paths=gta_train_path,
                    config=train_config,
                    ext=ext
                )
                self.gta_val = SailVosDataset(
                    paths=gta_val_path,
                    config=test_val_config,
                    ext=ext,
                )

        if stage == "test" or stage is None:
            # 테스트 셋은 기존 로직(txt 파일 기반)을 유지하거나,
            # 위에서 test_paths_set을 리스트로 변환해 넘겨줘도 됩니다.
            # 여기서는 편의상 기존 클래스가 is_test=True일 때 txt 파일을 읽는 로직을 그대로 쓴다고 가정하거나
            # 혹은 위와 똑같이 리스트를 만들어서 넘겨줍니다.

            # Test 리스트 생성 로직
            txt_path = self.config.celeb_df_path / "List_of_testing_videos.txt"
            test_df = pd.read_csv(txt_path, sep=" ", header=None, names=["label", "path"])
            test_videos = []
            if self.config.from_img:
                test_videos = [
                    str(self.config.celeb_df_path / Path(x).with_suffix('')).replace("\\", "/")
                    for x in test_df["path"].values
                ]
            else:
                test_videos = [str(self.config.celeb_df_path / Path(x)) for x in test_df["path"].values]
            # 실제 존재하는 폴더만 필터링
            test_videos = [v for v in test_videos if os.path.exists(v)]

            self.celebdf_test = self.dataset_cls(
                paths=test_videos,
                config=test_val_config,
            )
            if self.config.use_gta_v:
                assert gta_test_path is not None
                if not self.config.do_preprocess:
                    ext = ".bmp"
                else:
                    ext = ".png"
                self.gta_test = SailVosDataset(
                    paths=gta_test_path,
                    config=test_val_config,
                    ext=ext,
                )

    def train_dataloader(self):
        # 2. 현재 훈련 셋(Subset) 내의 클래스별 개수 계산
        label_counts = self.celebdf_train.get_label_counter()

        target_real_weight = 0.4
        target_fake_weight = 0.35
        target_gtav_weight = 0.25

        class_weights = (target_real_weight / label_counts[0],
                         target_fake_weight / label_counts[1])

        sample_weights = [class_weights[sample['label']]
                          for sample in self.celebdf_train.samples]

        dataset = self.celebdf_train

        if self.config.use_gta_v:
            dataset = ConcatDataset([dataset, self.gta_train])
            label_counts_gta = self.gta_train.get_label_counter()
            gtav_weight = target_gtav_weight / label_counts_gta[1]
            sample_weights.extend([gtav_weight] * len(self.gta_train))

        gen = None
        if self.config.seed is not None:
            gen = torch.Generator()
            gen.manual_seed(42)



        # 5. 샘플러 생성
        # num_samples는 보통 학습 데이터 전체 길이로 설정합니다.
        # replacement=True여야 불균형 데이터에서 적은 쪽을 중복해서 뽑아 균형을 맞춥니다.
        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=min(len(sample_weights), self.config.run_sample),
            replacement=True,
            generator=gen,
        )

        return DataLoader(
            dataset,
            **self._loader_param,
            sampler=sampler,
        )
    def val_dataloader(self):
        dataset = self.celebdf_val
        if self.config.use_gta_v:
            dataset = ConcatDataset([dataset, self.gta_val])
        return DataLoader(
            dataset,
            **self._loader_param,
        )
    def test_dataloader(self):
        dataset = self.celebdf_test
        if self.config.use_gta_v:
            dataset = ConcatDataset([dataset, self.gta_test])
        return DataLoader(
            dataset,
            **self._loader_param,
        )

# 학습

## Set wandb

In [31]:
import wandb
from google.colab import userdata
wandb_key = userdata.get('wandb_api')
wandb.login(key=wandb_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhnam0502 (dhnam0502-likelion) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Hyperparameter Search w/ Wandb Sweep

In [ ]:
import wandb
import lightning.pytorch as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torchvision.transforms import v2


# 1. Sweep 설정 정의
sweep_config = {
    'method': 'bayes',  # 이전 결과를 바탕으로 최적값을 찾아가는 베이지안 탐색
    'metric': {
        'name': 'val/MulticlassAveragePrecision',
        'goal': 'maximize'
    },
    'parameters': {
        # 우리가 집중적으로 조절할 하이퍼파라미터
        'lambda_1': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 1.5,
        }, # CE 비중
        'lambda_2': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 1.5,
        }, # AD 비중
        'delta_between': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 1.0
        }, # Head 간 거리 마진
        'delta_within_fake': {
            'distribution': 'uniform',
            'min': -2.0,
            'max': 0.5,
        }, # fake class에 대한 delta_within
        'delta_within_real': {
            'distribution': 'uniform',
            'min': -2.0,
            'max': 0.5,
        }, # real class에 대한 delta_within
        'eta': {
            'distribution': 'log_uniform_values',
            'min': 0.01,
            'max': 0.3,
        }, # Center 업데이트 속도
        'lr': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 5e-4,
        }, # Learning Rate

        # 고정할 값들 (실험 편의를 위해 여기에 선언)
        'max_epochs': {'value': 3},               # 싹수 확인을 위한 짧은 학습
        'batch_size': {'value': 20},
        'acc_grad': {'value': 2},
    }
}

# 2. Sweep ID 생성 (WandB 서버에 등록)
sweep_id = wandb.sweep(sweep_config, project="UNITE_hyper_tuning")
# sweep_id = 'dhnam0502-likelion/UNITE_hyper_tuning/t70rnjua'

LENGTH = 16
SIZE = (224, 224)


transform = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
])

# 3. 학습 실행 함수 정의
def sweep_train():
    # wandb.init()이 호출되면서 위에서 설정한 파라미터 중 하나를 가져옴
    with wandb.init() as run:
        config = wandb.config

        L.seed_everything(42, workers=True)

        # 이름 규칙 정하기 (대시보드에서 보기 편하게)
        run.name = f"L1_{config.lambda_1}_L2_{config.lambda_2}_fake_{config.delta_within_fake}"

        # 데이터 모듈 설정 (10,000개 샘플링 유지)
        # transform은 기존에 정의하신 것을 사용하세요.
        datamodule_config = DataModuleConfig(
            celeb_df_path = Path("/content/preprocessed"),
            from_img=True,
            use_gta_v=True,
            gta_v_path=Path("/content/preprocessed_gta"),
            seed=42,
            loader=DataLoaderConfig(
                batch_size=config.batch_size,
                num_workers=2,
                prefetch_factor=None,
                pin_memory=True,
                persistent_workers=True,
            ),
            dataset=DatasetConfig(
                num_frames=LENGTH,
                size=SIZE,
                encoder_model="google/siglip2-base-patch16-224",
            )
        )
        dm = DFDataModule(datamodule_config)
        # 중요: dm.setup()에서 사용하는 샘플링 로직은
        # train_dataloader의 WeightedRandomSampler가 처리합니다.

        # 모델 설정
        unite_classifier_config = UNITEClassifierConfig(
            num_frames=LENGTH,
            unite_model=UNITEConfig(
                dropout=0.1,
                encoder_model="google/siglip2-base-patch16-224",
                use_bfloat=True,
                cpu_preprocess=True,
            ),
            ad_loss=ADLossConfig(
                delta_within=(config.delta_within_real, config.delta_within_fake),
                delta_between=config.delta_between,
                eta=config.eta,
            ),
            optim=OptimizerConfig(
                lr=config.lr,
                decay_steps=(1000 * 32) // (config.batch_size * config.acc_grad)
            ),
            loss=LossConfig(
                lambda_1=config.lambda_1,
                lambda_2=config.lambda_2,
            )
        )
        model = LitUNITEClassifier(unite_classifier_config)

        # 모델 컴파일 (속도 향상)
        # model = torch.compile(model)

        # 로거 및 트레이너 설정
        logger = WandbLogger(log_model="all")

        trainer = L.Trainer(
            max_epochs=config.max_epochs,
            logger=logger,
            accumulate_grad_batches=config.acc_grad,
            precision='bf16-mixed',
            log_every_n_steps=50,
            num_sanity_val_steps=0, # 속도를 위해 생략
            deterministic=True,
        )

        # 학습 시작
        trainer.fit(model, datamodule=dm)

# 4. 에이전트 실행 (count=10은 총 10가지 조합을 시도한다는 뜻)
wandb.agent(sweep_id, function=sweep_train, count=20)

Epoch 2/2  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500/500 0:10:45 • 0:00:00 0.78it/s v_num: v6p2 val/loss: 0.228

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


epoch,▁▁▁▁▁▁▅▅▅▅▅▅██████
train/head_dist_mean,█▆▅▄▄▃▃▂▂▂▂▁▁▁▁
train/loss,██▃▃▁▂▂▁▃▁▁▃▁▃▃
train/loss_ad,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_ad/loss_between,▁▂▃▄▅▅▆▆▇▇▇▇███
train/loss_ad/loss_within,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
train/loss_ce,▄█▂▃▁▂▂▁▄▁▁▄▁▄▃
trainer/global_step,▁▁▂▃▃▃▃▄▅▅▅▅▆▇▇▇██
val/MulticlassAUROC,▁▆█
val/MulticlassAccuracy,▃▁█
+6,...


## Custom Run

### 학습 설정

In [33]:
from typing import Literal
import os
from torchvision.transforms import v2

BATCH_SIZE = 13
ACC_GRAD = 2
DECAY_STEPS = (1000 * 32) // (BATCH_SIZE * ACC_GRAD)
LENGTH = 16
SIZE_SIDE: Literal[224, 384] = 384
SIZE = (SIZE_SIDE, SIZE_SIDE)
ENCODER = f"google/siglip2-base-patch16-{SIZE_SIDE}"

# To test: (384, 384), 16 vs (224, 224), 32

transform = v2.Compose([
    v2.ToDtype(torch.uint8),
    v2.RandomHorizontalFlip(p=0.5),
    # v2.RandomApply([v2.RandomRotation((-10, 10))]), # Quite slow?
    v2.RandomApply([v2.GaussianBlur(kernel_size=(3, 7))]), # Managable...
    # v2.RandomApply([v2.ColorJitter(brightness=0.2, contrast=0.2)]), # Looks slow...
    v2.RandomApply([v2.JPEG((60, 100))]),
    v2.ToDtype(torch.float32),
])

datamodule_config = DataModuleConfig(
    celeb_df_path = Path("/content/preprocessed"),
    from_img=False,
    use_gta_v=True,
    do_preprocess=False,
    gta_v_path=Path("/content/preprocessed_gta"),
    loader=DataLoaderConfig(
        batch_size=BATCH_SIZE,
        num_workers=8,
        prefetch_factor=4,
        pin_memory=True,
        persistent_workers=True,
    ),
    dataset=DatasetConfig(
        num_frames=LENGTH,
        size=SIZE,
        encoder_model=ENCODER,
        transform=transform,
        device='cpu'
    )
)

datamodule = DFDataModule(datamodule_config)

unite_classifier_config = UNITEClassifierConfig(
    num_frames=LENGTH,
    unite_model=UNITEConfig(
        dropout=0.1,
        encoder_model=ENCODER,
        use_bfloat=True,
        cpu_preprocess=True,
    ),
    ad_loss=ADLossConfig(
        delta_within=(0.01, -2),
        delta_between=0.5,
    ),
    optim=OptimizerConfig(
        decay_steps=DECAY_STEPS
    ),
    loss=LossConfig(
        lambda_1=0.5,
        lambda_2=0.5,
    )
)

lit_classifier = LitUNITEClassifier(unite_classifier_config)


lit_classifier = torch.compile(lit_classifier)

Loading weights:   0%|          | 0/408 [00:00<?, ?it/s]

In [34]:
datamodule.prepare_data()

Using Colab cache for faster access to the 'celeb-df-v2' dataset.


In [35]:
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks.lr_monitor import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.profilers import AdvancedProfiler

profiler = AdvancedProfiler(".", "profile")


wandb_logger = WandbLogger(
    project="UNITE_deepfake_classification",
    name="with_gtav_loss_fix5_imgsz",
    # id="ocs3m68k",
    # resume="must",
)

# ckpt = ModelCheckpoint(monitor="val/acc", mode="max", save_last=True)
ckpt_drive = ModelCheckpoint(dirpath="/content/drive/MyDrive/bootcamp_proj/final/gtav_loss_fix5_imgsz/", monitor="val/MulticlassAveragePrecision", mode="max", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer =  L.Trainer(
    max_epochs=20,
    # max_steps=10,
    # profiler=profiler,
    logger=wandb_logger,
    callbacks=[ckpt_drive, lr_monitor],
    precision='bf16-mixed',
    log_every_n_steps=50,
    num_sanity_val_steps=0,
    accumulate_grad_batches=ACC_GRAD,
    # precision=16,
    fast_dev_run=True,
)


wandb_logger.watch(lit_classifier)


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:pytorch_lightning.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


### 학습

In [ ]:
trainer.fit(lit_classifier, datamodule=datamodule)

# trainer.fit(lit_classifier, datamodule=datamodule, ckpt_path='/content/drive/MyDrive/bootcamp_proj/final/gtav_loss_fix3_test2/last.ckpt')

In [37]:
trainer.checkpoint_callback.last_model_path

'/content/drive/MyDrive/bootcamp_proj/final/gtav_loss_fix5_imgsz/last-v1.ckpt'

In [38]:
trainer.checkpoint_callback.best_model_path

''

## 테스트

In [34]:
lit_classifier = LitUNITEClassifier.load_from_checkpoint('/content/drive/MyDrive/bootcamp_proj/final/gtav_loss_fix5_imgsz/last.ckpt', strict=False, weights_only=False)

Loading weights:   0%|          | 0/408 [00:00<?, ?it/s]

In [35]:
trainer.test(model=lit_classifier, datamodule=datamodule)

Using Colab cache for faster access to the 'celeb-df-v2' dataset.
GTA V data exists in gta-v/mini-ref-sailvos/Images, skipping preprocessing.
Processing 518 paths...
Loaded 6442 samples from 518 files/directories.
Processing 7 paths...
Loaded 42 samples from 7 files/directories.


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃           Test metric           ┃          DataLoader 0           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test/MulticlassAUROC       │       0.9499813318252563        │
│     test/MulticlassAccuracy     │       0.8757988214492798        │
│ test/MulticlassAveragePrecision │       0.9487425684928894        │
│    test/MulticlassPrecision     │       0.8632380962371826        │
│      test/MulticlassRecall      │       0.8757988214492798        │
└─────────────────────────────────┴─────────────────────────────────┘

[{'test/MulticlassAccuracy': 0.8757988214492798,
  'test/MulticlassAveragePrecision': 0.9487425684928894,
  'test/MulticlassPrecision': 0.8632380962371826,
  'test/MulticlassRecall': 0.8757988214492798,
  'test/MulticlassAUROC': 0.9499813318252563}]

## 학습 종료

In [36]:
wandb.finish()

epoch,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█
lr-AdamW,████▄▄▄▂▂▂▁▁▁▁
test/MulticlassAUROC,▁
test/MulticlassAccuracy,▁
test/MulticlassAveragePrecision,▁
test/MulticlassPrecision,▁
test/MulticlassRecall,▁
train/C_magnitude,▁▇██████████████████████████████████████
train/head_dist_mean,▁▂▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
train/loss,██▆▄▃▂▂▁▁▁▁▅▄▂▂▁▁▂▁▅▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+13,...
